In [1]:
import pandas as pd

In [13]:
df_1 = pd.read_csv('predicted_kaggle_2.csv')
df_2 = pd.read_csv('predicted_9883.csv')

In [14]:
df_1

,sign,filename
0,3.269911e-04,68f33844-472b-4111-b600-f90d544833c7
1,4.030051e-07,7d93a21d-1f16-49ce-8fcc-edf12c40f549
2,1.564724e-08,4a820650-7acd-489a-ad14-9d7ad8c73b6b
3,9.999995e-01,819b216b-2b6c-4539-a722-70648c0706c6
4,1.028908e-11,45f7c47d-03cc-40cd-acc5-b8c1c57872fa
...,...,...
100995,2.056264e-10,82aba9fd-3f78-4087-99f6-af9f5a990677
100996,1.994014e-11,3bb91d2b-53fb-42d4-a644-7e30a39572cd
100997,4.423170e-11,fba3e0de-d8b7-4721-82ae-f6ff135bd746
100998,8.033346e-11,09fa231e-6998-4904-acd7-2ed71408296e


In [15]:
df_2

,sign,filename
0,7.334746e-05,68f33844-472b-4111-b600-f90d544833c7
1,1.908693e-05,7d93a21d-1f16-49ce-8fcc-edf12c40f549
2,7.253241e-05,4a820650-7acd-489a-ad14-9d7ad8c73b6b
3,9.999462e-01,819b216b-2b6c-4539-a722-70648c0706c6
4,3.051769e-10,45f7c47d-03cc-40cd-acc5-b8c1c57872fa
...,...,...
100995,2.056264e-10,82aba9fd-3f78-4087-99f6-af9f5a990677
100996,1.994014e-11,3bb91d2b-53fb-42d4-a644-7e30a39572cd
100997,4.423170e-11,fba3e0de-d8b7-4721-82ae-f6ff135bd746
100998,8.033346e-11,09fa231e-6998-4904-acd7-2ed71408296e


In [8]:
df_full = df_2

In [11]:
df_full[1000:] = df_1[1000:]
df_full

,sign,filename
0,7.334746e-05,68f33844-472b-4111-b600-f90d544833c7
1,1.908693e-05,7d93a21d-1f16-49ce-8fcc-edf12c40f549
2,7.253241e-05,4a820650-7acd-489a-ad14-9d7ad8c73b6b
3,9.999462e-01,819b216b-2b6c-4539-a722-70648c0706c6
4,3.051769e-10,45f7c47d-03cc-40cd-acc5-b8c1c57872fa
...,...,...
100995,2.056264e-10,82aba9fd-3f78-4087-99f6-af9f5a990677
100996,1.994014e-11,3bb91d2b-53fb-42d4-a644-7e30a39572cd
100997,4.423170e-11,fba3e0de-d8b7-4721-82ae-f6ff135bd746
100998,8.033346e-11,09fa231e-6998-4904-acd7-2ed71408296e


In [12]:
df_full.to_csv('predicted_9883.csv', index=False)